<p style="text-align: center;font-size: 40pt">Rotation in 3D - Euler angles</p>

In [ ]:
%matplotlib widget
#%matplotlib inline

import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d

import numpy as np
import sympy as sp
from IPython.display import Math

import ipywidgets as widgets

%run ./scripts/helper_func.py
path = "{0}/lessons/transformations_3d/scripts/helper_func.py".format(get_root_path())
%run $path
path = "{0}/common/scripts/style.py".format(get_root_path())
%run $path

# Overview 

Requirements
- [Rotation matrix](2-lesson_rotation_mat.ipynb)

Objectives of this lesson:

- Give the basis of Euler angles
- Introduce the different conventions to apply rotations using Euler angles
- Introduce the concept of singularities and gimbal locks
- Give pros and cons for using Euler angles


Hidden custom latex commands here $ \curvearrowright$

----
[comment]: <> (General commands)
$\newcommand{\textcomma}{\quad\text{,}}$
$\newcommand{\textdot}{\quad\text{.}}$
$\newcommand{\vec}[1]{\overrightarrow{#1}}$
$\newcommand{\mat}[1]{\mathbf{#1}}$
$\newcommand{\frame}[1]{\mathcal{#1}}$
$\newcommand{\point}[2][]{{}^{#1}\mathbf{#2}}$
$\newcommand{\pointsym}[2][]{{}^{#1}\boldsymbol{#2}}$
$\newcommand{\matsym}[1]{\boldsymbol{#1}}$
$\newcommand{\real}{\mathbb{R}}$
$\newcommand{\bmat}[1]{\begin{bmatrix}#1\end{bmatrix}}$
$\newcommand{\F}[2][]{{}_{#2}^{#1}\mathscr{F}}$
$\newcommand{\Fmat}[2][]{{}_{#2}^{#1}\mat{F}}$
$\newcommand{\origin}[2][]{{}_{#2}^{#1}\mat{o}}$
$\newcommand{\T}[2][]{{}_{#2}^{#1}\mat{T}}$
$\newcommand{\t}[2][]{{}_{#2}^{#1}\mat{t}}$
$\newcommand{\R}[2][]{{}_{#2}^{#1}\mat{R}}$
$\newcommand{\f}{\vec{\mathscr{f}}}$
$\newcommand{\ax}[2][]{{}_{#2}^{#1}\vec{\mathscr{x}}}$
$\newcommand{\ay}[2][]{{}_{#2}^{#1}\vec{\mathscr{y}}}$
$\newcommand{\az}[2][]{{}_{#2}^{#1}\vec{\mathscr{z}}}$
$\newcommand{\aw}[2][]{{}_{#2}^{#1}\vec{\mathscr{w}}}$
$\newcommand{\axi}{\mathscr{x}}$
$\newcommand{\ayi}{\mathscr{y}}$
$\newcommand{\azi}{\mathscr{z}}$
$\newcommand{\awi}{\mathscr{w}}$
$\newcommand{\pointx}[2][]{{}^{#1}{#2}_{\axi}}$
$\newcommand{\pointy}[2][]{{}^{#1}{#2}_{\ayi}}$
$\newcommand{\pointz}[2][]{{}^{#1}{#2}_{\azi}}$
$\newcommand{\SO}[1]{\mathrm{SO}(#1)}$
----

# Parameters

The concept of Euler angle is relatively straightforward to understand.
Take your right hand and form a reference frame with your fingers.
Point your index finder in front of you and hold it with your other hand to rotate your frame by a quarter of a turn.
Then, hold your middle finger and turn by a negative quarter of a turn.
Finally, hold your thumb and turn by a quarter of a turn.
Your index should be pointing at the ceiling.
That is it, you just used Euler angles to apply a 3D rotation to you hand.
If there are people around you, they are now looking at you suspiciously...

<p style="text-align:center">
<img src="./images/CHF_200_9_front.jpg" width="75%" alt="Source:By Schweizer Nationalbank - Schweizer Nationalbank website, Public Domain, https://commons.wikimedia.org/w/index.php?curid=71634566"/>
<br>
Robotics gang sign directly on bills of 200 Swiss francs (~300 CAD).
I prefer using my index aligned with $\ax{}$, but I would still take the bill if someone gave it to me...
</p>

More formally, Euler angles are a sequence of three rotations around basis vectors.
The angles for those three rotations have many names depending of your application, but for this course we will use $\alpha$, $\beta$, and $\gamma $.
We can reach all possible rotations using a set of Euler angles defined as

\begin{aligned}
\matsym{\theta} = \left\{ \alpha, \beta, \gamma \right\}
\textcomma
\end{aligned}

where $\alpha \in [0, 2\pi]$, $\beta \in [0, \pi]$, and $\gamma  \in [0, 2\pi]$.
Of course, as we are talking about angles, one could center the range of the angles such that $\alpha \in [-\pi, \pi]$, $\beta \in [-\frac{\pi}{2}, \frac{\pi}{2}]$, and $\gamma  \in [-\pi, \pi]$, which would give the same coverage.
Now, the main problem comes when we need to agree on a convention to define which basis vectors are used and in which order.
Let's slowly go down the rabbit hole.


<p style="text-align:center">
<img src="./images/rabbit_hole.jpg" width="100%" />
</p>

## Intrinsic versus extrinsic

The two first categories are whether you are using _intrinsic rotations_ or  _extrinsic rotations_.
Intrinsic rotations are exactly what you did with your right hand in our first example.
Each angle is applied with respect to the newly rotated frame.
As for extrinsic rotations, each angle is applied with respect to a fixed frame.
To try it for yourself, you will need a pen and both of your hands.
Hold the pen with your left hand and align it with the index finger of your right hand.
Rotate the pen by a quarter of a turn around the thumb.
Then, by a quarter of a turn around the index finger.
Finally, rotate the pen by a negative quarter of a turn around your middle finger.
Your right hand was the fixed frame and the angles were always applied in relation with this frame.
Intrinsic rotations are the norm for vehicles and, as far as I know, extrinsic are sometimes used for robotic arms.
Luckily, every intrinsic rotation has an equivalent extrinsic rotation, so you can easily convert from one to another.
We will explain that conversion later and mainly focus on intrinsic rotations for now.

By only considering intrinsic rotations, we still have to fix the order of the basis vectors used, which still gives us 12 valid combinations.
It is getting so bad, that people came out with ways to encode the combinations.
A sequence where $\az{}$ is rotated by $\alpha$ radians, $\ay{}$ rotated by $\beta$ radians, and $\ax{}$ rotated by $\gamma$ radians can be simply written as the convention `z-y'-x''`.
Quotes `'` are also used to differentiate between extrinsic and intrinsic rotations.
The convention `z-y-x` means that basis vectors are fixed (i.e., extrinsic) and `z-y'-x"` means that basis vectors are rotated (i.e., intrinsic).

That being said, we can split our 12 combinations into two categories: _Proper Euler angles_ and _Tait–Bryan angles_.
Now you should be saying, _ark it was so easy when I was reading the joke about Swiss francs..._ and that's the point of this lesson.
As you train to be more knowledgeable on 3D transformations, it will be up to you to ask the proper questions about the convention used by people who randomly copy-paste code from Stack Overflow.
Trust me, I've been there.

## Proper Euler angles

To put that simply, this subset of conventions uses twice the same basis vector to define a rotation.
The six possible sequences are:
- `x-y'-x"`
- `x-z'-x"`
- `y-x'-y"`
- `y-z'-y"`
- `z-x'-z"`
- `z-y'-z"`

The easiest is to try it by yourself.
The following figure shows an example of the convention `z-y'-z"`, where each ring can be controlled with a slider.
You should be able to observe that: 
- changing the angle $\alpha$ rotates two smaller rings (i.e., the basis vectors `y'-z"`).
- changing the angle $\beta$ rotates the smaller blue ring (i.e., the basis vector `z"`).
- changing the angle $\gamma$ doesn't affect the other rings.

In [ ]:
%matplotlib widget
if 'fig1' in globals():
    plt.close(fig1)
    
fig1 = plt.figure(figsize=(4,4))


#------------------------
ax1 = fig1.add_subplot(111, projection="3d")
ax = ax1
ax.set_title(r"Proper Euler angles example")

# prepare handles
scat_zz = ax.scatter([],[],[], color="tab:blue", s=100, depthshade=False)
scat_y = ax.scatter([],[],[], color="tab:green", s=100, depthshade=False)
scat_z = ax.scatter([],[],[], color="tab:blue", s=100, depthshade=False)

h_circ_zz = ax.plot([], [], [], color="tab:blue", lw=2)
h_circ_y = ax.plot([], [], [], color="tab:green", lw=4)
h_circ_z = ax.plot([], [], [], color="tab:blue", lw=6)

f_x, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_y, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_z, _ = draw_3d_vector(ax, color="grey", alpha=0.75)


draw_3d_frame(ax, size=10)
ax.set_axis_off()
ax_lim = 1.
ax.set_xlim(-ax_lim, ax_lim); ax.set_ylim(-ax_lim, ax_lim); ax.set_zlim(-ax_lim, ax_lim)


radius_z = 1; radius_y = 0.9; radius_zz = 0.8

circle_z = build_cercle_xy(radius_z, 40)
circle_y = rotation_matrix_x(-np.pi/2.) @ build_cercle_xy(radius_y, 40)
circle_zz = build_cercle_xy(radius_zz, 40)

# points on ring
p_zz = radius_zz * np.array([[0],[1],[0]])
p_y = radius_y * np.array([[0, 0],[0, 0],[1, -1]])
p_z = radius_z * np.array([[1, -1],[0, 0],[0, 0]])

def update(alpha=0., beta=0.0, gamma=0.):
    R_z = rotation_matrix_z(alpha)
    R_y = rotation_matrix_y(beta)
    R_zz = rotation_matrix_z(gamma)
    
    p = R_z @ R_y @ R_zz @ p_zz
    scat_zz._offsets3d = p
    p = R_z @ R_y @ p_y
    scat_y._offsets3d = p
    p = R_z @ p_z
    scat_z._offsets3d = p
    
    R = R_z @ R_y @ R_zz
    f_x.set_positions(R[:,0])
    f_y.set_positions(R[:,1])
    f_z.set_positions(R[:,2])
    
    data_circ_zz = R_z @ R_y @ circle_zz
    data_circ_y = R_z @ circle_y
    
    list_circles = [(h_circ_zz, data_circ_zz),
                    (h_circ_y, data_circ_y),
                    (h_circ_z, circle_z)]
    
    for handle, data in list_circles:
        handle[0].set_xdata(data[0])
        handle[0].set_ydata(data[1])
        handle[0].set_3d_properties(zs=data[2])
    
    fig1.canvas.draw() # needed!


widgets.interact(update, alpha = (-np.pi, np.pi, 0.1), beta = (-np.pi/2., np.pi/2., 0.1), gamma = (-np.pi, np.pi, 0.1), continuous_update=False);

## Tait–Bryan angles

This subset of conventions uses all three axis to define a rotation.
The combinations are:
- `x-y'-z"`
- `x-z'-y"`
- `y-x'-z"`
- `y-z'-x"`
- `z-x'-y"`
- `z-y'-x"`

Those conventions are well suited to describe the orientation of a vehicle in space using [roll, pitch, and yaw](https://en.wikipedia.org/wiki/Aircraft_principal_axes).
Most of the time, but not always, the convention `z-y'-x"` is also known as nautical angles and could be considered the most probable convention.
But you should always triple check.

The following figure gives an example for the `z-y'-x"` convention.

In [ ]:
%matplotlib widget
if 'fig2' in globals():
    plt.close(fig2)
    
fig2 = plt.figure(figsize=(4,4))


#------------------------
ax1 = fig2.add_subplot(111, projection="3d")
ax = ax1
ax.set_title(r"Tait–Bryan angles example")

# prepare handles
scat_x = ax.scatter([],[],[], color="tab:red", s=100, depthshade=False)
scat_y = ax.scatter([],[],[], color="tab:green", s=100, depthshade=False)
scat_z = ax.scatter([],[],[], color="tab:blue", s=100, depthshade=False)

h_circ_x = ax.plot([], [], [], color="tab:red", lw=2)
h_circ_y = ax.plot([], [], [], color="tab:green", lw=4)
h_circ_z = ax.plot([], [], [], color="tab:blue", lw=6)

f_x, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_y, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_z, _ = draw_3d_vector(ax, color="grey", alpha=0.75)


draw_3d_frame(ax, size=10)
ax.set_axis_off()
ax_lim = 1.
ax.set_xlim(-ax_lim, ax_lim); ax.set_ylim(-ax_lim, ax_lim); ax.set_zlim(-ax_lim, ax_lim)


radius_z = 1; radius_y = 0.9; radius_x = 0.8

circle_z = build_cercle_xy(radius_z, 40)
circle_y = rotation_matrix_x(-np.pi/2.) @ build_cercle_xy(radius_y, 40)
circle_x = rotation_matrix_y(np.pi/2.) @ build_cercle_xy(radius_x, 40)

# points on ring
p_x = radius_x * np.array([[0],[1],[0]])
p_y = radius_y * np.array([[0, 0],[0, 0],[1, -1]])
p_z = radius_z * np.array([[1, -1],[0, 0],[0, 0]])

def update(alpha=0., beta=0., gamma=0.):
    R_z = rotation_matrix_z(alpha)
    R_y = rotation_matrix_y(beta)
    R_x = rotation_matrix_x(gamma)
    
    p = R_z @ R_y @ R_x @ p_x
    scat_x._offsets3d = p
    p = R_z @ R_y @ p_y
    scat_y._offsets3d = p
    p = R_z @ p_z
    scat_z._offsets3d = p
    
    R = R_z @ R_y @ R_x
    f_x.set_positions(R[:,0])
    f_y.set_positions(R[:,1])
    f_z.set_positions(R[:,2])
    
    data_circ_x = R_z @ R_y @ circle_x
    data_circ_y = R_z @ circle_y
    
    list_circles = [(h_circ_x, data_circ_x),
                    (h_circ_y, data_circ_y),
                    (h_circ_z, circle_z)]
    
    for handle, data in list_circles:
        handle[0].set_xdata(data[0])
        handle[0].set_ydata(data[1])
        handle[0].set_3d_properties(zs=data[2])
    
    fig2.canvas.draw() # needed!


widgets.interact(update, alpha = (-np.pi, np.pi, 0.1), beta = (-np.pi/2., np.pi/2., 0.1), gamma = (-np.pi, np.pi, 0.1), continuous_update=False);

# Convert Euler angles to rotation matrix

First, you need to recall principal rotations:

\begin{aligned}
\R{}_\axi (\theta)
= \bmat{ 
1 & 0 & 0
\\
0 & \cos \theta & -\sin \theta
\\ 
0 & \sin \theta & \cos \theta
\\
}
\quad
\R{}_\ayi (\theta)
= \bmat{ 
\cos \theta & 0 & \sin \theta
\\ 
0 & 1 & 0
\\
-\sin \theta & 0 & \cos \theta
\\
}
\quad
\R{}_\azi (\theta)
= \bmat{ 
\cos \theta & -\sin \theta & 0
\\
\sin \theta & \cos \theta & 0
\\ 
0 & 0 & 1
\\
}
\textdot
\end{aligned}

To produce a rotation matrix from an **intrinsic** rotation, you simply have to chain the principal rotation matrices associated to each basis vector.
The matrices will be built from the angles $\alpha$, $\beta$, and $\gamma$, such that

\begin{aligned}
\R{}_{\text{x-y'-x''}} &= \R{}_\axi (\alpha) \, \R{}_\ayi (\beta) \, \R{}_\axi (\gamma)
\\
\R{}_{\text{x-z'-x''}} &= \R{}_\axi (\alpha) \, \R{}_\azi (\beta) \, \R{}_\axi (\gamma)
\\
\R{}_{\text{y-x'-y''}} &= \R{}_\ayi (\alpha) \, \R{}_\axi (\beta) \, \R{}_\ayi (\gamma)
\\
\cdots &= \cdots
\\
\R{}_{\text{z-x'-y''}} &= \R{}_\azi (\alpha) \, \R{}_\axi (\beta) \, \R{}_\ayi (\gamma)
\\
\R{}_{\text{z-y'-x''}} &= \R{}_\azi (\alpha) \, \R{}_\ayi (\beta) \, \R{}_\axi (\gamma)
\\
\end{aligned}

It is a bit counter-intuitive, but you can also build an **extrinsic** rotation by chaining rotation matrices.
You simply need to inverse the order for the construction, thus

\begin{aligned}
\R{}_{\text{x-y-x}} &= \R{}_\axi (\gamma) \, \R{}_\ayi (\beta) \, \R{}_\axi (\alpha)
\\
\R{}_{\text{x-z-x}} &= \R{}_\axi (\gamma) \, \R{}_\azi (\beta) \, \R{}_\axi (\alpha)
\\
\R{}_{\text{y-x-y}} &= \R{}_\ayi (\gamma) \, \R{}_\axi (\beta) \, \R{}_\ayi (\alpha)
\\
\cdots &= \cdots
\\
\R{}_{\text{z-x-y}} &= \R{}_\ayi (\gamma) \, \R{}_\axi (\beta) \, \R{}_\azi (\alpha)
\\
\R{}_{\text{z-y-x}} &= \R{}_\axi (\gamma) \, \R{}_\ayi (\beta) \, \R{}_\azi (\alpha)
\\
\end{aligned}

One needs to be particularly careful with Tait–Bryan angles as the order of the principal rotation matrices also need to inverted, not only the angles.
This is also the case for Euler proper angles, but since it uses twice the same basis vector, one could easily oversee that detail.

The following cell implement principal rotation matrices using symbols instead of number to go deeper in our investigation without making too much mistakes in our equations.

In [ ]:
alpha = sp.Symbol('alpha')
beta = sp.Symbol('beta')
gamma = sp.Symbol('gamma')
c = sp.Function('\mathrm{c}')
s = sp.Function('\mathrm{s}')

def z(angle):
    return sp.Matrix([[ c(angle),-s(angle), 0],
                     [ s(angle), c(angle), 0],
                     [0, 0, 1]])
def y(angle):
    return sp.Matrix([[ c(angle), 0, s(angle)],
                     [0, 1, 0],
                     [-s(angle), 0, c(angle)]])

def x(angle):
    return sp.Matrix([[1, 0, 0],
                     [0, c(angle), -s(angle)],
                     [0, s(angle), c(angle)]])

## Proper Euler to rotation matrix

For a weird reason, virtually all lectures I found online will show what the full rotation matrix looks like in terms of the angles $\alpha$, $\beta$, and $\gamma$.
These matrices are so large that we need to use a short notation for trigonometry, such that $\cos() \mapsto \mathrm{c}()$ and $\sin() \mapsto \mathrm{s}()$.
I don't find it particularly useful, but let's play the game.
Those matrices are so prone to typo when writing them that I decided to use `sympy` to automate the process.

Nonetheless, here are the six rotation matrices for **intrinsic** rotations:

In [ ]:
xyx = x(alpha)*y(beta)*x(gamma)
xzx = x(alpha)*z(beta)*x(gamma)
yxy = y(alpha)*x(beta)*y(gamma)
yzy = y(alpha)*z(beta)*y(gamma)
zxz = z(alpha)*x(beta)*z(gamma)
zyz = z(alpha)*y(beta)*z(gamma)

display(Math(r"\mathbf{R}_\text{x-y'-x''} = " + sp.latex(xyx)))
print()
display(Math(r"\mathbf{R}_\text{x-z'-x''} = " + sp.latex(xzx)))
print()
display(Math(r"\mathbf{R}_\text{y-x'-y''} = " + sp.latex(yxy)))
print()
display(Math(r"\mathbf{R}_\text{y-z'-y''} = " + sp.latex(yzy)))
print()
display(Math(r"\mathbf{R}_\text{z-x'-z''} = " + sp.latex(zxz)))
print()
display(Math(r"\mathbf{R}_\text{z-y'-z''} = " + sp.latex(zyz)))
print()

Fun right? At least now you know.

## Tait–Bryan angles to rotation matrix

The same automated process was used here, with the final rotation matrix in green being the nautical angles (i.e., roll, pitch, yaw), which might be more popular than the others.

In [ ]:
xyz = x(alpha)*y(beta)*z(gamma)
xzy = x(alpha)*z(beta)*y(gamma)
yxz = y(alpha)*x(beta)*z(gamma)
yzx = y(alpha)*z(beta)*x(gamma)
zxy = z(alpha)*x(beta)*y(gamma)
zyx = z(alpha)*y(beta)*x(gamma)

display(Math(r"\mathbf{R}_\text{x-y'-z''} = " + sp.latex(xyz)))
print()
display(Math(r"\mathbf{R}_\text{x-z'-y''} = " + sp.latex(xzy)))
print()
display(Math(r"\mathbf{R}_\text{y-x'-z''} = " + sp.latex(yxz)))
print()
display(Math(r"\mathbf{R}_\text{y-z'-x''} = " + sp.latex(yzx)))
print()
display(Math(r"\mathbf{R}_\text{z-x'-y''} = " + sp.latex(zxy)))
print()
display(Math(r"\color{green}{\mathbf{R}_\text{z-y'-x''} = " + sp.latex(zyx) + '}'))


**Pro tip**: knowing how to build a rotation matrix from principal rotations is useful to avoid implementation errors and to quickly change from one standard to another, but it is not the most efficient way to implement it.
Once everything is stable and you are in optimization phase, you should convert it using a single matrix instead of a chain of three rotation matrices.
The following benchmark shows the kind of gain you could expect.

In [ ]:
# Euler parameters
alpha=1.6; beta = 0.8; gamma = 2.3

# a point
p = np.array([[2],[1],[6]])

# ouch, not fun to code, and easy to make a typo...
def R_zyx(alpha, beta, gamma):
    return np.array([[np.cos(alpha)*np.cos(beta), -np.sin(alpha)*np.cos(gamma) + np.sin(beta)*np.sin(gamma)*np.cos(alpha), np.sin(alpha)*np.sin(gamma) + np.sin(beta)*np.cos(alpha)*np.cos(gamma)], 
                    [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma) + np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma) - np.sin(gamma)*np.cos(alpha)], 
                    [-np.sin(beta), np.sin(gamma)*np.cos(beta), np.cos(beta)*np.cos(gamma)]
                   ])

# build three principal rotation matrix
R_z = rotation_matrix_z(alpha)
R_y = rotation_matrix_y(beta)
R_x = rotation_matrix_x(gamma)

# build a single matrix
R = R_zyx(alpha, beta, gamma)

print("With three principal rotations:")
score1 = %timeit -o p_prime = R_z @ R_y @ R_x @ p
p_prime = R_z @ R_y @ R_x @ p
display(Math(r"p'=" + sp.latex(sp.Matrix(p_prime))))

print()
print("With one messy matrix:")
score2 = %timeit -o p_prime = R @ p
p_prime = R @ p
display(Math(r"p'=" + sp.latex(sp.Matrix(p_prime))))

print("Improvement of %.2f%% on computation time." % (((score1.best-score2.best)/score1.best)*100))

# Singularities

As for any 3D rotation parametrization using only three parameters, there are singularities with Euler angles.
Those singularities even have a fancy name: gimbal locks.
Nowadays, a [gimbal](https://en.wikipedia.org/wiki/Gimbal) is known as the mechanical system stabilizing a camera, but before it was also how we built gyroscopes.
Their construction produces the same sequence of rotations than Euler angles.

A singularity happens when we loose a degree of freedom in the parameters.
This means that two of the parameters produce the same rotation.
For Tait–Bryan angles, this singularity happens when $\beta = \left\{-\frac{\pi}{2}, \frac{\pi}{2} \right\}$
For proper Euler angles, this singularity happens when $\beta = 0$.
To experiment a gimbal lock, go back to the first interactive plot of the [proper Euler angles section](#Proper-Euler-angles) and make sure that $\beta = 0$.
Then, play with $\alpha$ and $\gamma$.
You will be able to achieve the same rotation with both parameters.
A good video showing a physical gimbal can be seen [here](https://youtu.be/oj7v3MXJL3M?t=11).

**Extra**: Gimbals were used in early spaceships to measure orientation.
If a pilot would hit a gimbal lock, the orientation would be lost.
Now, you should be better equipped to understand this scene from the [movie Apollo 13](https://youtu.be/C3J1AO9z0tA?t=126).
Closer to robotics, gimbal locks also happen on robotic arms and are known as [wrist singularities](https://youtu.be/lD2HQcxeNoA).


# Interpolation

You can go ahead and execute the next cell.
In the following figure, we have a point in yellow that we want to rotate to a target location marked with an `x`.
We want to know what are the small rotations leading to this final large rotation.
One way of doing this is by linearly interpolating each Euler angle.
This is useful in computer games, but also when you want to control a robot by giving a few keyframes and having a trajectory being generated by interpolation.
The optimal rotation between the original point and the rotated point should be an arc of circle.
Before playing with the slider of the figure, do the following:
1. freely rotate the camera to understand the scene
1. align the `x` with the point
1. use the slider to move the point along the trajectory
1. freely rotate the camera to understand the trajectory

What went wrong?


In [ ]:
%matplotlib widget
if 'fig3' in globals():
    plt.close(fig3)
    
fig3 = plt.figure(figsize=(4,4))

# a point
p = np.array([[-0.3],[1.2],[0]])

res = 30
l_alpha = np.linspace(0, np.pi/3., res)
l_beta = np.linspace(0, np.pi/6., res)
l_gamma = np.linspace(0, 8*np.pi/6., res)
l_R = []
l_p = np.empty((3,res))

for i in range(res):
    R_z = rotation_matrix_z(l_alpha[i])
    R_y = rotation_matrix_y(l_beta[i])
    R_x = rotation_matrix_x(l_gamma[i])
    R = R_z @ R_y @ R_x
    p_prime = R @ p
    l_R.append(R)
    l_p[:,i] = p_prime.flatten()
    
#------------------------
ax1 = fig3.add_subplot(111, projection="3d")
ax = ax1
ax.set_title(r"Interpolation of Euler angles")

# prepare handles
current_p = ax.scatter([],[],[], color="yellow", s=60, depthshade=False)
target_p = ax.scatter(l_p[0,-1], l_p[1,-1], l_p[2,-1], marker='x', color="yellow", s=100, depthshade=False)


h_traj = ax.plot([], [], [], color="yellow", lw=2)

f_x, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_y, _ = draw_3d_vector(ax, color="grey", alpha=0.75)
f_z, _ = draw_3d_vector(ax, color="grey", alpha=0.75)


draw_3d_frame(ax, size=10)
ax.set_axis_off()
ax_lim = 1.
ax.set_xlim(-ax_lim, ax_lim); ax.set_ylim(-ax_lim, ax_lim); ax.set_zlim(-ax_lim, ax_lim)

def update(index=0):
    
    R = l_R[index]
    f_x.set_positions(R[:,0])
    f_y.set_positions(R[:,1])
    f_z.set_positions(R[:,2])
    
    h_traj[0].set_xdata(l_p[0,0:index])
    h_traj[0].set_ydata(l_p[1,0:index])
    h_traj[0].set_3d_properties(zs=l_p[2,0:index])
    
    current_p._offsets3d = l_p[:,index]
    
    fig3.canvas.draw() # needed!


widgets.interact(update, index = (0, res-1, 1), continuous_update=False);

The interpolation is not too bad if the keyframes are tightly spaced, but can cause problem for large angles.
Imagine you are controlling a quadrotor that needs to navigate around trees to reach a certain location.
Having a large deviation from an arc of circle will most probably have your quadrotor collide with trees very soon.
You can also observe the same behavior with a robotic arm connecting two key locations with a large angle offset.

# Operations

Well there is none!
Although $\matsym{\theta} = \left\{ \alpha, \beta, \gamma \right\}$ looks like a vector (i.e., it is not, it's a set), you cannot not add or subtract two of them and expect a meaningful result.
You need to convert the sets of angles to rotation matrices and then use the operations rotation matrices support to rotate points or chain with other transformations.


# Pros and cons

- Easy to understand at first.

- Easy to anticipate what rotation will be produced by using your right hand and applying sequentially the rotations. 

- Prone to hidden typos when converting to a rotation matrix.

- A lot of conventions to choose from.
  Even when knowing that it is most probably a nautical angles convention, one could still be confused on whether it was `z-y'-x"` or `x-y'-z"`.
  While reading to write this lesson, I stumbled upon sentences such as _"In the first edition of our book, we used the convention `x-y'-z"`, but for the second edition we changed to `z-y'-x"` "_.
  
- Singularities (i.e., gimbal locks) must be actively avoided.

- Unexpected trajectories of points when rotations are interpolated.

- There is no operation to rotate points. 
  They are mainly used to build a rotation matrix.



# Conclusion

You should do the following activities to enhance your understanding of the concepts viewed in this lesson:
- play with the Python scripts provided;
- do the [exercises](../../exercises/ex_transformations_3d/3e-exercises_euler_angles.ipynb) related to this lesson, they are necessary to connect concepts;
- modify the markdown by adding your own notes using `> my notes`; and
- complete the tables [Symbol definitions](#Symbol-definitions) and [Glossary](#Glossary) and add your own definitions.

Parallel lesson:
- [Axis-angle](3-lesson_rotation_axis_angle.ipynb)

Next lesson:
- [Special Euclidean group](5-lesson_se3.ipynb)

## Symbol definitions

| Symbol                        | Definition                          |
|--------------------           |-------------                        |
| $\R{}$                        | rotation matrix                     |
| $r_\axi$, $r_\ayi$, $r_\azi$  | each column of the rotation matrix  |
| $\SO{3}$                      | special orthogonal group            |
| ...                           |                                     |

## Glossary

| English             | Français                    | Definition |
|-----------          |------------                 |------------|
| extrinsic rotation  | rotation extrinsèque        |            |
| intrinsic rotation  | rotation intrinsèque        |            |
| proper Euler angles | angles originaux d'Euler    |            |
| Tait–Bryan angles   | angles Tait-Bryan           |            |
| roll                | roulis                      |            |
| pitch               | tangage                     |            |
| yaw                 | lacet                       |            |
| gimbal lock         | blocage de cardan           |            |
| nautical angles     | angles nautiques            |            |
| ...                 |                             |            |